# **MLOps- MODELO DE RECOMENDACIÓN DE PELICULAS**

El modelo consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score de similaridad y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función adicional de la API anterior y debe llamarse:

- def recomendacion( titulo ): Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

In [168]:
#Librerias necesarias
import pandas as pd
import numpy as np 
# importamos el modulo de sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

# **1. Preparación de los datos**

In [169]:
#Creamos el df con los datos de las peliculas
movies = pd.read_csv('movies_final.csv')
#Mostramos la info del df
movies.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45451 entries, 0 to 45450
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   45451 non-null  int64  
 1   title                45451 non-null  object 
 2   genre                45451 non-null  object 
 3   collection           45451 non-null  object 
 4   original_language    45440 non-null  object 
 5   spoken_language      45451 non-null  object 
 6   runtime              45205 non-null  float64
 7   release_date         45451 non-null  object 
 8   release_year         45451 non-null  int64  
 9   director_name        44617 non-null  object 
 10  companies            45451 non-null  object 
 11  production_countrie  45451 non-null  object 
 12  status               45371 non-null  object 
 13  overview             44510 non-null  object 
 14  popularity           45451 non-null  float64
 15  vote_average         45451 non-null 

- Para el modelo de recomendación se usarán aquellas columnas que en el EDA mostraron ciertas caracteristicas relevantes. Es por ello que se seleccionaran de acuerdo a un criterio de evaluación personal

**1.1 Selección de columnas**

In [170]:
#Determinamos las columnas iniciales
columnas = ['title', 'genre', 'director_name', 'overview', 'companies', 'vote_average', 'release_year']

#Creamos el df
df = movies[columnas]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45451 entries, 0 to 45450
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          45451 non-null  object 
 1   genre          45451 non-null  object 
 2   director_name  44617 non-null  object 
 3   overview       44510 non-null  object 
 4   companies      45451 non-null  object 
 5   vote_average   45451 non-null  float64
 6   release_year   45451 non-null  int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 2.4+ MB


**1.2 Tratamiento de los datos**

- Para mejorar el performance del modelo y teniendo en cuenta la disponibilidad de consumo de las peliculas, se tomaran unicamente los peliculas producidas de 1970 en adelante. 

In [171]:
#Creamos mascara
mask = df['release_year'] >= 1970

#Definimos el nuevo df utilizando la mascara
df = df[mask]

- Tratamos las peliculas donde haya datos nulos en las columnas requeridas 

In [172]:
#Rellenamos los valores NaN de las columnas overview y director 
df['overview'] = df['overview'].fillna("")
df['director_name'] = df['director_name'].fillna("")

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37150 entries, 0 to 45450
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          37150 non-null  object 
 1   genre          37150 non-null  object 
 2   director_name  37150 non-null  object 
 3   overview       37150 non-null  object 
 4   companies      37150 non-null  object 
 5   vote_average   37150 non-null  float64
 6   release_year   37150 non-null  int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 2.3+ MB


- Eliminamos la columna release_year y normalizamos la columna vote_average


In [174]:
#Eliminamos la columna release_year
df.drop(columns='release_year', inplace= True)

# Normalización de 'vote_average' para que esté en el rango [0, 1]
df['vote_average_normalized'] = (df['vote_average'] - df['vote_average'].min()) / (df['vote_average'].max()
                                                                                    - df['vote_average'].min())
#Borramos la columna 'vote_average'
df.drop(columns='vote_average', inplace= True)

In [175]:
#Verificamos
df.sample(2)

,title,genre,director_name,overview,companies,vote_average_normalized
2465,10 Things I Hate About You,"Comedy, Romance, Drama",Gil Junger,"Bianca, a tenth grader, has never gone on a da...","Mad Chance, Jaret Entertainment, Touchstone Pi...",0.73
13641,The End Of America,Documentary,,"Based on her book of the same name, Naomi Wolf...",None,0.73


- Quitamos la coma a la columna 'genre' y de 'companie'

In [176]:
# Modificamos la columna con géneros combinados
df['genre'] = df['genre'].str.replace(', ', ' ')

In [177]:
#Modificamos la columna 'companie'
df['companies'] = df['companies'].str.replace(', ', ' ')

In [178]:
df.sample(2)

,title,genre,director_name,overview,companies,vote_average_normalized
18782,"Jeff, Who Lives at Home",Drama Comedy,Jay Duplass,Dispatched from his basement room on an errand...,Indian Paintbrush,0.61
15184,The Losers,Action Adventure Crime Mystery Thriller,Sylvain White,"A tale of double cross and revenge, centered u...",DC Comics Weed Road Pictures Dark Castle Enter...,0.62


- Reducimos el dataset en función a las votos promedios para obtener "las mejores peliculas".

In [179]:
vote_mask = df['vote_average_normalized'] >= 0.60
df = df[vote_mask]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19445 entries, 0 to 45447
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    19445 non-null  object 
 1   genre                    19445 non-null  object 
 2   director_name            19445 non-null  object 
 3   overview                 19445 non-null  object 
 4   companies                19445 non-null  object 
 5   vote_average_normalized  19445 non-null  float64
dtypes: float64(1), object(5)
memory usage: 1.0+ MB


- Creación de columna para perfiles, estandarización de formato y eliminación de columnas finales 

In [180]:
# Creación de perfiles de películas
df['features'] = df['genre'] + ' ' + df['companies'] #+ ' ' + df['overview']

# Rellenamos con '' los valores NaN de ser existir.  
df['features'] = df['features'].fillna('')

# Convertimos en minusculas los valores de la columna features
df['features'] = df['features'].str.lower()

In [181]:
#Borramos duplicados en función al titulo de las peliculas 
df = df.drop_duplicates(subset='title')

In [182]:
df.sample(3)

,title,genre,director_name,overview,companies,vote_average_normalized,features
24664,The Ballad of Johnny Guitar,Drama,João César Monteiro,Lord knows where John of God has been. He's co...,Madragoa Filmes Grupo de Estudos e Realizações...,0.63,drama madragoa filmes grupo de estudos e reali...
40628,In the Shadow of Death,Drama Adventure,Gunārs Piesis,"A group of men go under-ice fishing, but find ...",Rigas Kinostudija,0.65,drama adventure rigas kinostudija
16117,127 Hours,Adventure Drama Thriller,Danny Boyle,The true story of mountain climber Aron Ralsto...,Fox Searchlight Pictures Warner Bros. Cloud Ei...,0.70,adventure drama thriller fox searchlight pictu...


In [183]:
#Eliminamos las columnas innecesarias 
df.drop(columns=['genre','companies','overview','director_name'], inplace= True)

In [184]:
#Revisamos el dataset final 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18777 entries, 0 to 45447
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    18777 non-null  object 
 1   vote_average_normalized  18777 non-null  float64
 2   features                 18777 non-null  object 
dtypes: float64(1), object(2)
memory usage: 586.8+ KB


In [190]:
#Reseteamos el indice
df = df.reset_index(drop=True)

In [192]:
df.tail()

,title,vote_average_normalized,features
18772,Mom,0.66,crime drama thriller mad films third eye pictures
18773,St. Michael Had a Rooster,0.60,none none
18774,Shadow of the Blair Witch,0.70,mystery horror none
18775,The Burkittsville 7,0.70,horror neptune salad entertainment pirie produ...
18776,Century of Birthing,0.90,drama sine olivia


# **2. Creación del Modelo de recomendación** 

**2.1 Vectorización de características**
- Para el modelo se usará 'TfidfVectorizer' ya que se desea conservar la semántica de las palabras y considerar la importancia relativa.

In [193]:
#Inicializamos Tfidfvectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
#Transformamos los datos 
tfidf_matrix = tfidf_vectorizer.fit_transform(df['features'])

In [194]:
# imprimimos el tamaño de la matrix
tfidf_matrix.shape

(18777, 11362)

**2.2 Construcción de matriz final**
- La idea original era considerar el promedio de votos de cada pelicula para calcular la similitud de coseno, sin embargo el procesamiento requeria mayores recursos, siendo especificos una mayor cantidad de memoria RAM disponible. 
- De todas maneras se deja el codigo comentado. 

In [19]:
# Incorporación de 'vote_average_normalized' en la matriz de características
# vote_average_normalized_array = df['vote_average_normalized'].values.reshape(-1, 1)
# tfidf_matrix_with_vote = pd.concat([pd.DataFrame(tfidf_matrix.toarray()), pd.DataFrame(vote_average_normalized_array)], axis=1)

**2.3 Modelamiento**

- Calculo de similitud de coseno

In [195]:
# Cálculo de la matriz similitud coseno
similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

- Indice de peliculas

In [196]:
# Creamos un índice de películas 
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [197]:
similarities.shape

(18777, 18777)

- Función de recomendación basada en similitud

In [198]:
# Función de recomendación basada en similitud
def recomendacion(title):
    """Función que recibe de argumento una pelicula y recomienda las 5 con mayor similitud"""
    #Indice de la pelicula
    movie_index = indices[title]

    #Creamos una lista de tuplas donde cada tupla contiene el índice de la película y 
    # su similitud con la película seleccionada.
    similar_movies = list(enumerate(similarities[movie_index]))

    #Ordenamos la lista similar_movies en orden descendente
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    #Seleccionamos las 5 primeras
    top_similar = similar_movies[1:6]

    # Obtenemos los índices de las películas
    movie_indices = [i[0] for i in top_similar]

    #Creamos una lista de las peliculas
    #peliculas_recomendadas = df_highly_rated['title'].iloc[movie_indices].tolist()
    peliculas_recomendadas = df['title'].iloc[movie_indices].tolist()
    return peliculas_recomendadas

- Ejemplos de recomendación 

In [199]:
pelicula = 'Toy Story'
recomendaciones = recomendacion(pelicula)
print(f"Ya que viste '{pelicula}' te recomiendo ver: {recomendaciones}")

Ya que viste 'Toy Story' te recomiendo ver: ['Toy Story 2', "Dug's Special Mission", 'Finding Nemo', 'Partly Cloudy', 'BURN·E']


In [200]:
pelicula = 'Kurt Cobain: About a Son'
recomendaciones = recomendacion(pelicula)
print(f"Ya que viste '{pelicula}' te recomiendo ver: {recomendaciones}")

Ya que viste 'Kurt Cobain: About a Son' te recomiendo ver: ['Beautiful Losers', 'Message to Love: The Isle of Wight Festival', 'Buena Vista Social Club', 'The Decline of Western Civilization Part II: The Metal Years', 'Down from the Mountain']


In [201]:
pelicula = 'Dreams of Dust'
recomendaciones = recomendacion(pelicula)
print(f"Ya que viste '{pelicula}' te recomiendo ver: {recomendaciones}")

Ya que viste 'Dreams of Dust' te recomiendo ver: ['Lamerica', 'Under The Domim Tree', 'Kaspar Hauser', 'Voyage to the Beginning of the World', 'The Dreamlife of Angels']


**CONCLUSIONES DEL MODELO**
- El modelo final basa sus recomendaciones en función al género y la productora de la pelicula.
- Las recomendaciones en su mayoria son cercanas a lo que se podría esperar. 
- Se descarto overview por temas de procesamiento, capacidad de memoria y por que cuando fue ejecutada en conjunto con género no necesariamente devolvia peliculas muy relacionadas. 

-------------------------------------

# **Creación de DATASET para DEPLOY**

- Debido a que la capacidad de render es limitada, se encontro como solución crear un dataset que contenga el titulo de las peliculas y las 5 recomendaciones que brinda el modelo.
- **ACLARACIÓN**: Esta solución funciona solo en casos de contener un dataset original fijo, osea que no recibe cargas continuas.

**1. Creación de df completo las 5 recomendaciones**

In [202]:
# Crear una lista para almacenar los datos de recomendaciones
recomendaciones_data = []

# Iterar a través de las películas en el DataFrame
for movie_title in df['title']:
    
    # Obtener recomendaciones para la película
    recomendadas = recomendacion(movie_title)  
    
    #Creamos diccionario con cada pelicula y sus recomendaciones
    recomendaciones_data.append({'Pelicula': movie_title, 'Recomendaciones': recomendadas})

# Crear un DataFrame a partir de la lista de datos de recomendaciones
recomendaciones_df = pd.DataFrame(recomendaciones_data)

In [203]:
# Crear un DataFrame a partir de la lista de datos de recomendaciones
recomendaciones_df = pd.DataFrame(recomendaciones_data)

In [206]:
recomendaciones_df.head(20)

,Pelicula,Recomendaciones
0,Toy Story,"[Toy Story 2, Dug's Special Mission, Finding N..."
1,Jumanji,"[Blind Fury, Pitch Black, The Associate, Snow ..."
2,Grumpier Old Men,"[Grumpy Old Men, You've Got Mail, Summer of '4..."
3,Waiting to Exhale,"[Ever After: A Cinderella Story, How Stella Go..."
4,Heat,"[A Time to Kill, The Negotiator, The Sunchaser..."
5,Sabrina,"[Mother, The Truman Show, Major League, The Ra..."
6,GoldenEye,"[For Your Eyes Only, The Living Daylights, Liv..."
7,The American President,"[Forget Paris, Othello, City Hall, Beyond Rang..."
8,Balto,"[An American Tail: Fievel Goes West, An Americ..."
9,Nixon,"[Tombstone, Deep Rising, The Joy Luck Club, My..."


- Exportamos el df creado a un archivo csv para ser consumido por la API

In [205]:
recomendaciones_df.to_csv('recomendaciones.csv', index= False)

**2. Creación de función de recomendación para API**

- Ya que se tiene un dataset hecho, la función para obtener las 5 recomendaciones es solo un filtro

In [ ]:
# ML
@app.get('/recomendacion/{titulo}')
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    return {'lista recomendada': respuesta}

In [207]:
df_modelo = pd.read_csv('recomendaciones.csv')

In [208]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18777 entries, 0 to 18776
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Pelicula         18777 non-null  object
 1   Recomendaciones  18777 non-null  object
dtypes: object(2)
memory usage: 293.5+ KB


In [209]:
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    #Creamos mascara para buscar la pelicula 
    recomendaciones = df_modelo[df_modelo['Pelicula'] == titulo]['Recomendaciones'].values[0]
    return {'lista recomendada': recomendaciones}

In [210]:
recomendacion('Kurt Cobain: About a Son')

{'lista recomendada': "['Beautiful Losers', 'Message to Love: The Isle of Wight Festival', 'Buena Vista Social Club', 'The Decline of Western Civilization Part II: The Metal Years', 'Down from the Mountain']"}

-----------------------------------------------------------